## <p style="text-align: center;">Fine-Tuning a Faster Convolutional Neural Network for Infrared Drone Image thermal Signature Classification in Wildlife Conservation</p>

<p style="text-align: center;">Abstract: </p>
<p style="text-align: center;"> This tutorial demonstrates how to effectively fine tune and evaluate a Faster CNN model using the birdsai dataset of night time infrared images from drones in Southern Africa. The tutorial starts with how to best set up the data for the fine tuning, then the process of fine-tuning and evaluating the Faster CNN model. The resulting model is one that can be used to classify thermal signatures identified by drones at night time as either being that of a humnan or animal, which can support anit-poaching and conservation efforts. </p>


 By completing this tutorial, you will be able to:

- Apply transfer learning concepts to thermal imagery classification
- Implement CNN fine-tuning techniques using PyTorch
- Understand the impact of hyperparameters on model accuracy
- Evaluate model performance for conservation applications

## Table of Contents
1. [Introduction](#introduction)
2. [Environment Setup](#setup)
3. [Data Preparation](#data)
4. [Model Architecture](#model)
5. [Training Process](#training)
6. [Hyperparameter Exploration](#hyperparameters)
7. [Evaluation](#evaluation)
8. [Results and Discussion](#results)
9. [References](#references)

<a id="introduction"></a>
## 1. Introduction

Wildlife conservation increasingly relies on thermal imaging drones to monitor protected areas. Distinguishing humans (potential poachers) from animals presents unique challenges due to similar heat signatures. This tutorial applies transfer learning with pre-trained CNNs for this specialized classification task.

Unlike existing tutorials that focus on standard RGB images, our approach addresses thermal imagery's unique characteristics while integrating systematic experiment tracking for reproducibility.

<a id="setup"></a>
## 2. Environment Setup

In [ ]:
# Import libraries
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import json
import cv2
from PIL import Image

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# For evaluation
from sklearn.metrics import confusion_matrix, classification_report

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed) 
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed()


# Define device as CPU explicitly
device = torch.device("cpu")
print(f"Using device: {device}")